In [1]:
%run ..\notebooks\Util_func.ipynb

In [2]:
# # Bay Area
# REG = 'BayArea'
# base_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\Processing_20200228\2_tour_extract\wt_wkday'
# raw_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\Processing_20200228\spatial_join'

# out_file = r'out\%s_11_Person_Attr.xlsx' %REG

In [3]:
# # SANDAG
# REG = 'SANDAG'
# base_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\SANDAG\2_tour_extract\wt_wkday'
# raw_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\SANDAG'

# out_file = r'out\%s_11_Person_Attr.xlsx' %REG

In [4]:
# SCAG
REG = 'SCAG'
base_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\SCAG_dataset_2020-02-27\2_tour_extract\wt_wkday'
raw_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\SCAG_dataset_2020-02-27'

out_file = r'out\%s_11_Person_Attr.xlsx' %REG

In [5]:
## Process household records
hh_df = pd.read_csv(join(raw_dir, 'ex_hh_wZones.csv'))
hh_df = hh_df[['hh_id','home_county_fips']]
## Process person records
tmp_df = pd.read_csv(join(raw_dir, 'ex_person_wZones.csv'))
tmp_df = tmp_df.merge(hh_df, how='left')
if REG=='SANDAG' or REG=='SCAG':
    tmp_df2 = pd.read_csv(join(raw_dir, 'ex2_person.tsv'), sep='\t')
    tmp_df = tmp_df.merge(tmp_df2[['hh_id','person_id','raceeth_new_imputed']], how='left')
tmp_df = tmp_df[['hh_id','person_num','raceeth_new_imputed','income_imputed','gender','age','home_county_fips']]
tmp_df = tmp_df.rename(columns={'raceeth_new_imputed':'raceeth','income_imputed':'hinc'})
tmp_df = tmp_df.rename(columns={'hh_id':'hhno','person_num':'pno'})

per_df = pd.read_csv(join(base_dir,'survey2018_precx.dat'), sep=' ')
per_df = per_df.merge(tmp_df, how='left')
per_df['count'] = 1
per_df = per_df[per_df['psexpfac']>0]

per_df.loc[per_df['gender']==997, 'gender'] = 5 #Other
per_df.loc[per_df['gender']==999, 'gender'] = 6 #NoAnswer
per_df.loc[per_df['gender'].isin([-9998, 995]), 'gender'] = 6 #Missing

In [6]:
col_dict = {
    'raceeth': {
        'desc': 'RaceEth',
        'col': 'raceeth',
        'vals': range(1,6),
        'labels': ['1_Hispanic','2_Black','3_Asian/PI','4_White','5_Other']
    },
    'hinc': {
        'desc': 'HHInc',
        'col': 'hinc',
        'vals': range(1,9),
        'labels': ['1_25K','2_25_50K','3_50_75K','4_75_100K','5_100_150K','6_150_200K','7_200_250K','8_250K']
    },
    'age': {
        'desc': 'Age',
        'col': 'age',
        'vals': range(4,11),
        'labels': ['18-24','25-34','35-44','45-54','55-64','65-74','75+']
    },
    'gender': {
        'desc': 'Gend',
        'col': 'gender',
        'vals': range(1,7),
        'labels': ['1_F','2_M','3_Trns','4_NBin','5_Oth','6_Miss']
    }
}
if REG=='SANDAG':
    col_dict['home_county'] = {
        'desc': 'home_county',
        'col': 'home_county_fips',
        'vals': [73],
        'labels': ['San_Diego']
    }
elif REG=='SCAG':
    col_dict['home_county'] = {
        'desc': 'home_county',
        'col': 'home_county_fips',
        'vals': [37,59],
        'labels': ['1_LA', '2_Orange']
    }
else:
    col_dict['home_county'] = {
        'desc': 'home_county',
        'col': 'home_county_fips',
        'vals': [1,13,41,55,75,81,85,95,97],
        'labels': ['4_Alameda', '5_CCosta', '9_Marin', '7_Napa', '1_SF', '2_SanMateo', '3_SC', '6_Solano', '8_Sonoma']
    }

In [7]:
fname = out_file
writer = pd.ExcelWriter(fname, engine='xlsxwriter')
workbook = writer.book
format1 = workbook.add_format({'num_format': '#,##0.0'})

In [8]:
from xlsxwriter.utility import xl_rowcol_to_cell

wt_cols = ['count', 'psexpfac']
wt_desc = ['(Unweighted)', '(Weighted)']

d1_dict = col_dict['home_county']
for key in ['raceeth', 'hinc', 'age', 'gender']:
    d2_dict = col_dict[key]
    row = 0
    sname = d2_dict['desc']
    
    for wc, wd in zip(wt_cols, wt_desc):
        title = 'Persons by ' + d2_dict['desc'] + ' and ' + d1_dict['desc'] + ' ' + wd
        tab, tab_fmt = prep_data_2d(per_df,d1_dict['col'],d1_dict['vals'],d1_dict['labels'],
                                d2_dict['col'],d2_dict['vals'],d2_dict['labels'],wc)
        row = write_to_excel(tab.astype('float64'), sname, title, row)

        title = 'Row Shares by ' + d2_dict['desc'] + ' and ' + d1_dict['desc'] + ' ' + wd
        row = write_to_excel(getRowSharesIdx(tab.copy()), sname, title, row)

    tab_range = xl_rowcol_to_cell(row,1) + ':' + xl_rowcol_to_cell(row,tab.shape[1])
    _ = writer.sheets[sname].set_column(tab_range, 11, format1)   

In [9]:
writer.save()